###Preparation

In [6]:
#load the file and see the descriptive
from google.colab import drive
drive.mount('/content/drive')
HOME_DIR = '/content/drive/My Drive/Colab Notebooks/dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pickle
%matplotlib inline

In [18]:
#open file tweet

df = pd.read_csv(HOME_DIR+'/data_capres.csv',  encoding="ISO-8859-1")

In [19]:
df

,ID,Tweet_ID,Isi_Tweet,Tanggal_Tweet,Kandidat,Aplikasi,Lokasi User,Sentimen
0,133343972,patih_patria,@FahrelStv Gak setuju Jokowi jadi Cawapres.. C...,8/1/2013 0:33,Jokowi,web,BLITAR - INDONESIA,1
1,921720487,RintisBautista,"capres jokowi,wacapres abraham samad. gubernur...",8/1/2013 4:11,Jokowi,"<a href=""http://janetter.net/"" rel=""nofollow"">...",Didalam hatinya recha :love,1
2,-2120434840,JokowiPres,"RT @RintisBautista: capres jokowi,wacapres abr...",8/1/2013 4:32,Jokowi,"<a href=""http://roundteam.co"" rel=""nofollow"">R...",NaN,1
3,-116006035,Franliiiii,capres ( Prabowo ) and cawapres ( jokowi ) and...,8/1/2013 7:33,Prabowo,web,Manado,1
4,-906300614,JokowiPres,RT @Franliiiii: capres ( Prabowo ) and cawapre...,8/1/2013 7:36,Prabowo,"<a href=""http://roundteam.co"" rel=""nofollow"">R...",NaN,1
...,...,...,...,...,...,...,...,...
3351,1908041164,Ino8687,@TrioMacan2000 tuh Can ..capres Wiranto menyam...,2/28/2014 8:45,None,"<a href=""http://blackberry.com/twitter"" rel=""n...",NaN,-1
3352,-318754400,WENRYap,"Jokowi (Presiden),Risma (Wakil Presiden),Megaw...",2/28/2014 10:51,None,"<a href=""https://about.twitter.com/products/tw...",NaN,1
3353,-372183855,IrlindaTama,RT @UtamaDiana: Caper kelesRT @ratnaaydn: Ini ...,2/28/2014 13:50,None,"<a href=""http://store.ovi.com/content/256340"" ...",NaN,-1
3354,509204748,TDHIPANDA88,@Prabowo08 maju untuk THE NEXT presiden.prabow...,2/28/2014 14:53,None,"<a href=""http://twitter.com/download/android"" ...",NaN,1


In [21]:
df_use = df[['Isi_Tweet', 'Sentimen']]
df_use.head()

,Isi_Tweet,Sentimen
0,@FahrelStv Gak setuju Jokowi jadi Cawapres.. C...,1
1,"capres jokowi,wacapres abraham samad. gubernur...",1
2,"RT @RintisBautista: capres jokowi,wacapres abr...",1
3,capres ( Prabowo ) and cawapres ( jokowi ) and...,1
4,RT @Franliiiii: capres ( Prabowo ) and cawapre...,1


In [22]:
#load stopwords

stp = open(HOME_DIR+'twitter_stp.dic')
stp_words = []
item = stp.readline()

while item != '':
  stp_words.append(item)
  item = stp.readline()

In [23]:
stp_words

['a\n',
 'ada\n',
 'adalah\n',
 'ah\n',
 'akan\n',
 'aku\n',
 'all\n',
 'an\n',
 'and\n',
 'apa\n',
 'are\n',
 'at\n',
 'atas\n',
 'atau\n',
 'b\n',
 'bagi\n',
 'bang\n',
 'banget\n',
 'baru\n',
 'be\n',
 'best\n',
 'bgt\n',
 'biar\n',
 'bikin\n',
 'bisa\n',
 'blm\n',
 'boleh\n',
 'bro\n',
 'bs\n',
 'but\n',
 'by\n',
 'can\n',
 'cc\n',
 'cek\n',
 'co\n',
 'cont\n',
 'cuma\n',
 'd\n',
 'dah\n',
 'dalam\n',
 'dan\n',
 'dari\n',
 'day\n',
 'deh\n',
 'dengan\n',
 'depan\n',
 'dgn\n',
 'di\n',
 'dia\n',
 'dimana\n',
 'do\n',
 'donk\n',
 'dong\n',
 'dr\n',
 'dulu\n',
 'dunk\n',
 'eh\n',
 'emang\n',
 'follow\n',
 'for\n',
 'from\n',
 'g\n',
 'ga\n',
 'gimana\n',
 'gitu\n',
 'gua\n',
 'gue\n',
 'gw\n',
 'haha\n',
 'hahaha\n',
 'hanya\n',
 'have\n',
 'hehe\n',
 'hehehe\n',
 'http\n',
 'i\n',
 "i'm\n",
 'in\n',
 'ini\n',
 'is\n',
 'it\n',
 'itu\n',
 'iya\n',
 'iye\n',
 'jd\n',
 'jg\n',
 'jgn\n',
 'juga\n',
 'just\n',
 'k\n',
 'ka\n',
 'kak\n',
 'kalau\n',
 'kali\n',
 'kalian\n',
 'kalo\n',
 'kam

## Preprocessing

In [24]:
import re
import string
from textblob import TextBlob

list_data = []

for i in df_use.Isi_Tweet:
  list_data.append(i)

list_data_bersih1 = []
list_data_bersih = []




In [25]:
list_data

['@FahrelStv Gak setuju Jokowi jadi Cawapres.. CAPRES JOKOWI harga mati...',
 'capres jokowi,wacapres abraham samad. gubernur ahok. gua yakin koruptor abissss !',
 'RT @RintisBautista: capres jokowi,wacapres abraham samad. gubernur ahok. gua yakin koruptor abissss !',
 'capres ( Prabowo ) and cawapres ( jokowi ) and gubDKI (Ahok) mantap lanjutkan !!! buat pak presiden (SBY) bubarkan saja FPI',
 'RT @Franliiiii: capres ( Prabowo ) and cawapres ( jokowi ) and gubDKI (Ahok) mantap lanjutkan !!! buat pak presiden (SBY) bubarkan saja FPI',
 'jd skenarionya gini. 2014 biar Prabowo jd presiden, Jokowi tetepgubernur. kalo Jakarta berhasil gak usah nunggu 2019 buat gantiin Prabowo',
 'SBY mantan TNI, dan Calon Presiden Prabowo subianto adalah Mantan KOPASSUS.. Krn anggoto TNI disiplin.. Maka dari itu, smw presiden TEGAS!!!',
 '@Prabowo08 mohon untuk pak Presiden Prabowo Subiyanto, tolong jangan jadikan kami "Perangkat Desa" sebagai tumbal kebijakan pemerintah.',
 'RT @gusti_primus: SBY mantan T

In [27]:
import re
pat1 = r'@[A-Za-z0-9_]+' #deleting @mention
pat2 = r'https?://[A-Za-z0-9./]+' #delete url
pat3 = r'#[A-Za-z0-9]+' #delete hashtag
pat4 = r'\[A-Za-z0-9]+'#delete alphabeth+number
pat5 = r'\<[A-Za-z0-9+_>]+' #deleting error mp
pat6 = r'[0-9]+' #delete full of number with more than 1
pattern = re.compile('\W')  #deleting white space

combined_pat = r'|'.join((pat1, pat2, pat3,pat4, pat5, pat6)) #masking creating

for j in list_data:
  dataBersih1 = re.sub(combined_pat,'',str(j)) #filtering process
  list_data_bersih1.append(dataBersih1) #store cleaning data


In [28]:
for k in list_data_bersih1:   
  dataBersih2 = re.sub(pattern,' ',str(k)) #menghapus karakter non alphanumeric
  dataBersih3 = re.sub(r'\b\w{1,1}\b', '', dataBersih2) #mengapus karakter 1 digit
  list_data_bersih.append(dataBersih3) #menyimpan dalam list

list_data_bersih


[' Gak setuju Jokowi jadi Cawapres   CAPRES JOKOWI harga mati   ',
 'capres jokowi wacapres abraham samad  gubernur ahok  gua yakin koruptor abissss  ',
 'RT   capres jokowi wacapres abraham samad  gubernur ahok  gua yakin koruptor abissss  ',
 'capres   Prabowo   and cawapres   jokowi   and gubDKI  Ahok  mantap lanjutkan     buat pak presiden  SBY  bubarkan saja FPI',
 'RT   capres   Prabowo   and cawapres   jokowi   and gubDKI  Ahok  mantap lanjutkan     buat pak presiden  SBY  bubarkan saja FPI',
 'jd skenarionya gini   biar Prabowo jd presiden  Jokowi tetepgubernur  kalo Jakarta berhasil gak usah nunggu  buat gantiin Prabowo',
 'SBY mantan TNI  dan Calon Presiden Prabowo subianto adalah Mantan KOPASSUS   Krn anggoto TNI disiplin   Maka dari itu  smw presiden TEGAS   ',
 ' mohon untuk pak Presiden Prabowo Subiyanto  tolong jangan jadikan kami  Perangkat Desa  sebagai tumbal kebijakan pemerintah ',
 'RT   SBY mantan TNI  dan Calon Presiden Prabowo subianto adalah Mantan KOPASSUS   Kr

In [29]:
# feature extraction using Bag of Words
vect = TfidfVectorizer(min_df=0.005, max_df=0.99, stop_words=stp_words, use_idf=True, ngram_range=(1,3), smooth_idf=False, norm='l2')

In [30]:
data_sentiment = []

for i in df_use.Sentimen:
  data_sentiment.append(i)

tabel = list(zip(list_data_bersih, data_sentiment))
tabel_frame = pd.DataFrame(tabel, columns=['tweet', 'label'])
tabel_frame.head()


,tweet,label
0,Gak setuju Jokowi jadi Cawapres CAPRES JOKO...,1
1,capres jokowi wacapres abraham samad gubernur...,1
2,RT capres jokowi wacapres abraham samad gub...,1
3,capres Prabowo and cawapres jokowi and...,1
4,RT capres Prabowo and cawapres jokowi ...,1


In [31]:
tabel_frame.shape

(3356, 2)

## Splitting Dataset

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_label = train_test_split(tabel_frame.tweet, tabel_frame.label, train_size=0.75)

In [33]:
X_train.shape

(2517,)

In [34]:
y_train.shape

(2517,)

##Modelling Part : 1st Attempt

In [38]:
#Multinomial db
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

model = MultinomialNB()

model1_pipeline = Pipeline([
                     ('vect',vect),
                     ('clf',model)
])

In [39]:
model1_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaa', 'aaaa', 'aaah', 'aah', 'aan', 'ad', 'ada', 'adalah', 'adalh', 'adlah', 'adlh', 'aga', 'agak', 'agar', 'agk', 'agr', 'ah', 'ahaha', 'ahh', 'ahhh', 'ai', 'aj', 'aja', 'ajaa', 'ajah', 'aje', 'ak', 'akan', 'akhirnya', 'akhrny', 'akhrnya', 'akibat', 'akibatnya', 'akn', 'ako', 'aku', 'all', 'als', 'amat', 'ampe', 'amt', 'an', 'and', 'andaikan', 'andaikata', 'andaikn', 'ane', 'ank', 'antar', 'antara', 'antaranya', 'antarny', 'antr', 'antra', 'antrany', 'ap', 'apa', 'apaa', 'apabila', 'apabla', 'apakah', 'apalagi', 'apbl', 'ape', 'are', 'as', 'asal', 'asalkan', 'at', 'atas', 'atau', 'ataupun', 'ato', 'ava', 'aw', 'ay', 'ba', 'bagai', 'bagaikan', 'bagaimana', 'bagi', 'bahkan', 'bahwa', 'baik', 'bang', 'banget', 'banyak', 'barangkali', 'baru', 'be', 'bebas', 'beberapa', 'begitu', 'belum', 'berap

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.99, max_features=None,
                                 min_df=0.005, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=False,
                                 stop_words=['a\n', 'ada\n', 'adalah\n', 'ah\n',
                                             'akan\n', 'aku\n', 'al...
                                             'atas\n', 'atau\n', 'b\n',
                                             'bagi\n', 'bang\n', 'banget\n',
                                             'baru\n', 'be\n', 'best\n',
                                             'bgt\n', 'biar\n', 'bikin\n',
                       

In [40]:
y_pred = model1_pipeline.predict(X_test)
print(classification_report(y_pred, y_label)) #percobaan1

              precision    recall  f1-score   support

          -1       0.46      0.69      0.55       136
           0       0.77      0.67      0.72       422
           1       0.66      0.63      0.64       281

    accuracy                           0.66       839
   macro avg       0.63      0.66      0.64       839
weighted avg       0.68      0.66      0.67       839



In [47]:
#gridsearch tuning param 1

# grid search
from sklearn.model_selection import GridSearchCV
params = { 'vect__min_df' : [0, 0.05, 0.1 ],
          'vect__max_df' : [0.7, 0.8, 0.9, 0.95, 0.99, 1],
          'clf__alpha' : [0.01, 0.02, 0.08, 0.09, 0.1, 0.15, 0.2,  0.5, 0.6] }

grid = GridSearchCV(model1_pipeline, params, n_jobs=4, cv=5)

In [48]:
grid.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaa', 'aaaa', 'aaah', 'aah', 'aan', 'ad', 'ada', 'adalah', 'adalh', 'adlah', 'adlh', 'aga', 'agak', 'agar', 'agk', 'agr', 'ah', 'ahaha', 'ahh', 'ahhh', 'ai', 'aj', 'aja', 'ajaa', 'ajah', 'aje', 'ak', 'akan', 'akhirnya', 'akhrny', 'akhrnya', 'akibat', 'akibatnya', 'akn', 'ako', 'aku', 'all', 'als', 'amat', 'ampe', 'amt', 'an', 'and', 'andaikan', 'andaikata', 'andaikn', 'ane', 'ank', 'antar', 'antara', 'antaranya', 'antarny', 'antr', 'antra', 'antrany', 'ap', 'apa', 'apaa', 'apabila', 'apabla', 'apakah', 'apalagi', 'apbl', 'ape', 'are', 'as', 'asal', 'asalkan', 'at', 'atas', 'atau', 'ataupun', 'ato', 'ava', 'aw', 'ay', 'ba', 'bagai', 'bagaikan', 'bagaimana', 'bagi', 'bahkan', 'bahwa', 'baik', 'bang', 'banget', 'banyak', 'barangkali', 'baru', 'be', 'bebas', 'beberapa', 'begitu', 'belum', 'berap

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.99,
                                                        max_features=None,
                                                        min_df=0.005,
                                                        ngram_range=(1, 3),
                                                     

In [49]:
grid.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.7, max_features=None,
                                 min_df=0, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=False,
                                 stop_words=['a\n', 'ada\n', 'adalah\n', 'ah\n',
                                             'akan\n', 'aku\n', 'all\n',...
                                             'atas\n', 'atau\n', 'b\n',
                                             'bagi\n', 'bang\n', 'banget\n',
                                             'baru\n', 'be\n', 'best\n',
                                             'bgt\n', 'biar\n', 'bikin\n',
                       

In [45]:
#implement vectorizer
vect = TfidfVectorizer(min_df=0, max_df=0.7, stop_words=stp_words, use_idf=True, ngram_range=(1,3), smooth_idf=False, norm='l2')
model_multi = MultinomialNB(alpha=0.08)

model1_tuning_pipeline = Pipeline([
                     ('vect',vect),
                     ('clf',model_multi)
])

model1_tuning_pipeline.fit(X_train, y_train)
y_pred = model1_tuning_pipeline.predict(X_test)
print(classification_report(y_pred, y_label))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaa', 'aaaa', 'aaah', 'aah', 'aan', 'ad', 'ada', 'adalah', 'adalh', 'adlah', 'adlh', 'aga', 'agak', 'agar', 'agk', 'agr', 'ah', 'ahaha', 'ahh', 'ahhh', 'ai', 'aj', 'aja', 'ajaa', 'ajah', 'aje', 'ak', 'akan', 'akhirnya', 'akhrny', 'akhrnya', 'akibat', 'akibatnya', 'akn', 'ako', 'aku', 'all', 'als', 'amat', 'ampe', 'amt', 'an', 'and', 'andaikan', 'andaikata', 'andaikn', 'ane', 'ank', 'antar', 'antara', 'antaranya', 'antarny', 'antr', 'antra', 'antrany', 'ap', 'apa', 'apaa', 'apabila', 'apabla', 'apakah', 'apalagi', 'apbl', 'ape', 'are', 'as', 'asal', 'asalkan', 'at', 'atas', 'atau', 'ataupun', 'ato', 'ava', 'aw', 'ay', 'ba', 'bagai', 'bagaikan', 'bagaimana', 'bagi', 'bahkan', 'bahwa', 'baik', 'bang', 'banget', 'banyak', 'barangkali', 'baru', 'be', 'bebas', 'beberapa', 'begitu', 'belum', 'berap

              precision    recall  f1-score   support

          -1       0.68      0.80      0.74       174
           0       0.83      0.77      0.80       392
           1       0.77      0.77      0.77       273

    accuracy                           0.78       839
   macro avg       0.76      0.78      0.77       839
weighted avg       0.78      0.78      0.78       839

              precision    recall  f1-score   support

          -1       0.68      0.80      0.74       174
           0       0.83      0.77      0.80       392
           1       0.77      0.77      0.77       273

    accuracy                           0.78       839
   macro avg       0.76      0.78      0.77       839
weighted avg       0.78      0.78      0.78       839



/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaa', 'aaaa', 'aaah', 'aah', 'aan', 'ad', 'ada', 'adalah', 'adalh', 'adlah', 'adlh', 'aga', 'agak', 'agar', 'agk', 'agr', 'ah', 'ahaha', 'ahh', 'ahhh', 'ai', 'aj', 'aja', 'ajaa', 'ajah', 'aje', 'ak', 'akan', 'akhirnya', 'akhrny', 'akhrnya', 'akibat', 'akibatnya', 'akn', 'ako', 'aku', 'all', 'als', 'amat', 'ampe', 'amt', 'an', 'and', 'andaikan', 'andaikata', 'andaikn', 'ane', 'ank', 'antar', 'antara', 'antaranya', 'antarny', 'antr', 'antra', 'antrany', 'ap', 'apa', 'apaa', 'apabila', 'apabla', 'apakah', 'apalagi', 'apbl', 'ape', 'are', 'as', 'asal', 'asalkan', 'at', 'atas', 'atau', 'ataupun', 'ato', 'ava', 'aw', 'ay', 'ba', 'bagai', 'bagaikan', 'bagaimana', 'bagi', 'bahkan', 'bahwa', 'baik', 'bang', 'banget', 'banyak', 'barangkali', 'baru', 'be', 'bebas', 'beberapa', 'begitu', 'belum', 'berap

In [51]:
#implement vectorizer
vect = TfidfVectorizer(min_df=0, max_df=0.7, stop_words=stp_words, use_idf=True, ngram_range=(1,3), smooth_idf=False, norm='l2')
model_multi = MultinomialNB(alpha=0.01)

model1_tuning_pipeline = Pipeline([
                     ('vect',vect),
                     ('clf',model_multi)
])

model1_tuning_pipeline.fit(X_train, y_train)
y_pred = model1_tuning_pipeline.predict(X_test)
print(classification_report(y_pred, y_label))

              precision    recall  f1-score   support

          -1       0.75      0.76      0.75       201
           0       0.81      0.80      0.81       366
           1       0.77      0.77      0.77       272

    accuracy                           0.78       839
   macro avg       0.78      0.78      0.78       839
weighted avg       0.78      0.78      0.78       839



/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaa', 'aaaa', 'aaah', 'aah', 'aan', 'ad', 'ada', 'adalah', 'adalh', 'adlah', 'adlh', 'aga', 'agak', 'agar', 'agk', 'agr', 'ah', 'ahaha', 'ahh', 'ahhh', 'ai', 'aj', 'aja', 'ajaa', 'ajah', 'aje', 'ak', 'akan', 'akhirnya', 'akhrny', 'akhrnya', 'akibat', 'akibatnya', 'akn', 'ako', 'aku', 'all', 'als', 'amat', 'ampe', 'amt', 'an', 'and', 'andaikan', 'andaikata', 'andaikn', 'ane', 'ank', 'antar', 'antara', 'antaranya', 'antarny', 'antr', 'antra', 'antrany', 'ap', 'apa', 'apaa', 'apabila', 'apabla', 'apakah', 'apalagi', 'apbl', 'ape', 'are', 'as', 'asal', 'asalkan', 'at', 'atas', 'atau', 'ataupun', 'ato', 'ava', 'aw', 'ay', 'ba', 'bagai', 'bagaikan', 'bagaimana', 'bagi', 'bahkan', 'bahwa', 'baik', 'bang', 'banget', 'banyak', 'barangkali', 'baru', 'be', 'bebas', 'beberapa', 'begitu', 'belum', 'berap

In [67]:
# dump pipeline to pickle
pickle.dump(model1_tuning_pipeline,open(HOME_DIR+'model1_riko-wijayanto.pkl', 'wb'))

## Modelling Part : 2nd Attempt

In [58]:
#initialize random forest

from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier()

# feature extraction using Bag of Words
vect_temp = TfidfVectorizer(min_df=0.005, max_df=0.99, stop_words=stp_words, use_idf=True, ngram_range=(1,3), smooth_idf=False, norm='l2')

model2_pipeline = Pipeline([
                     ('vect_2',vect_temp),
                     ('clf_2',model2)
])


In [59]:
model2_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaa', 'aaaa', 'aaah', 'aah', 'aan', 'ad', 'ada', 'adalah', 'adalh', 'adlah', 'adlh', 'aga', 'agak', 'agar', 'agk', 'agr', 'ah', 'ahaha', 'ahh', 'ahhh', 'ai', 'aj', 'aja', 'ajaa', 'ajah', 'aje', 'ak', 'akan', 'akhirnya', 'akhrny', 'akhrnya', 'akibat', 'akibatnya', 'akn', 'ako', 'aku', 'all', 'als', 'amat', 'ampe', 'amt', 'an', 'and', 'andaikan', 'andaikata', 'andaikn', 'ane', 'ank', 'antar', 'antara', 'antaranya', 'antarny', 'antr', 'antra', 'antrany', 'ap', 'apa', 'apaa', 'apabila', 'apabla', 'apakah', 'apalagi', 'apbl', 'ape', 'are', 'as', 'asal', 'asalkan', 'at', 'atas', 'atau', 'ataupun', 'ato', 'ava', 'aw', 'ay', 'ba', 'bagai', 'bagaikan', 'bagaimana', 'bagi', 'bahkan', 'bahwa', 'baik', 'bang', 'banget', 'banyak', 'barangkali', 'baru', 'be', 'bebas', 'beberapa', 'begitu', 'belum', 'berap

Pipeline(memory=None,
         steps=[('vect_2',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.99, max_features=None,
                                 min_df=0.005, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=False,
                                 stop_words=['a\n', 'ada\n', 'adalah\n', 'ah\n',
                                             'akan\n', 'aku\n', '...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                

In [60]:
y_pred = model2_pipeline.predict(X_test)
print(classification_report(y_pred, y_label))

              precision    recall  f1-score   support

          -1       0.68      0.85      0.76       164
           0       0.86      0.76      0.81       410
           1       0.75      0.77      0.76       265

    accuracy                           0.78       839
   macro avg       0.76      0.79      0.77       839
weighted avg       0.79      0.78      0.78       839



In [63]:
model2 = RandomForestClassifier(min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0)

# feature extraction using Bag of Words
vect_temp = TfidfVectorizer(min_df=0, max_df=0.7, stop_words=stp_words, use_idf=True, ngram_range=(1,3), smooth_idf=False, norm='l2')

model2_pipeline = Pipeline([
                     ('vect_2',vect_temp),
                     ('clf_2',model2)
])

In [64]:
model2_pipeline.fit(X_train, y_train)
y_pred = model1_tuning_pipeline.predict(X_test)
print(classification_report(y_pred, y_label))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aaa', 'aaaa', 'aaah', 'aah', 'aan', 'ad', 'ada', 'adalah', 'adalh', 'adlah', 'adlh', 'aga', 'agak', 'agar', 'agk', 'agr', 'ah', 'ahaha', 'ahh', 'ahhh', 'ai', 'aj', 'aja', 'ajaa', 'ajah', 'aje', 'ak', 'akan', 'akhirnya', 'akhrny', 'akhrnya', 'akibat', 'akibatnya', 'akn', 'ako', 'aku', 'all', 'als', 'amat', 'ampe', 'amt', 'an', 'and', 'andaikan', 'andaikata', 'andaikn', 'ane', 'ank', 'antar', 'antara', 'antaranya', 'antarny', 'antr', 'antra', 'antrany', 'ap', 'apa', 'apaa', 'apabila', 'apabla', 'apakah', 'apalagi', 'apbl', 'ape', 'are', 'as', 'asal', 'asalkan', 'at', 'atas', 'atau', 'ataupun', 'ato', 'ava', 'aw', 'ay', 'ba', 'bagai', 'bagaikan', 'bagaimana', 'bagi', 'bahkan', 'bahwa', 'baik', 'bang', 'banget', 'banyak', 'barangkali', 'baru', 'be', 'bebas', 'beberapa', 'begitu', 'belum', 'berap

              precision    recall  f1-score   support

          -1       0.75      0.76      0.75       201
           0       0.81      0.80      0.81       366
           1       0.77      0.77      0.77       272

    accuracy                           0.78       839
   macro avg       0.78      0.78      0.78       839
weighted avg       0.78      0.78      0.78       839



In [66]:
# dump pipeline to pickle
pickle.dump(model2_pipeline,open(HOME_DIR+'model2_riko-ijayanto.pkl', 'wb'))